In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

import os
from os import listdir
from os.path import isfile, join

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import OPTICS
from sklearn.mixture import BayesianGaussianMixture 
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Reshape, Dense, BatchNormalization, Dropout, Flatten
from tensorflow.keras.layers import MaxPooling1D,MaxPooling2D,AveragePooling1D
from tensorflow.keras.layers import Activation, LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv1D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

import keras_radam
from keras_radam import RAdam
from tensorflow.keras.utils import plot_model

import numpy as np
import joblib 
import matplotlib.pyplot as plt

from CRISMrelatedMethods.preprocessing import *
from CRISMrelatedMethods.dataRead import *
from CRISMrelatedMethods.score import *

In [2]:
augmentedData_X=joblib.load('AugmentedPrep_X.z')
augmentedData_Y=joblib.load('AugmentedPrep_Y.z')
labelledData_X=joblib.load('plebaniMICA_testPrep_X.z')
labelledData_Y=joblib.load('plebaniMICA_testPrep_Y.z')

In [3]:
def fitCNN(trainingData,Ydata,batch_size=240, epochs=50,prep=''):
    trainingData_rs = trainingData.reshape(trainingData.shape[0],trainingData.shape[1],1)
    X_train,X_val,y_train,y_val = train_test_split(trainingData_rs,Ydata,test_size=0.25,random_state=20)
    newYTrain = to_categorical(y_train)
    newYval = to_categorical(y_val)

    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu',input_shape=(trainingData.shape[1],1)))
    model.add(MaxPooling1D(pool_size=(3), strides=(1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Conv1D(filters=64, kernel_size=5, strides=1, activation='relu'))
    model.add(MaxPooling1D(pool_size=(5), strides=(2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Conv1D(filters=64, kernel_size=9, strides=1, activation='relu'))
    model.add(MaxPooling1D(pool_size=(9), strides=(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(newYTrain.shape[1], activation='softmax'))

    loss='categorical_crossentropy'
    optimizer = RAdam(total_steps=5000, warmup_proportion=0.2, min_lr=1e-5)
    metrics=['accuracy']
    model.compile(loss=loss, optimizer=optimizer , metrics=metrics)
    joblib.dump({'RAdam':optimizer},'models/CNN_optimizer_'+prep+'.z')

    checkpoint=ModelCheckpoint('models/CNN_'+prep+'.h5',monitor='val_loss',verbose=0,save_best_only=True,save_weights_only=False,mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=50)
    model.fit(X_train, newYTrain, validation_data=(X_val, newYval), epochs=epochs,batch_size = batch_size, verbose=0, callbacks=[es,checkpoint]) 
    
def fitANN(trainingData,Ydata,batch_size=300, epochs=75,prep=''):
    trainingData_rs = trainingData.reshape(trainingData.shape[0],trainingData.shape[1])
    X_train,X_val,y_train,y_val = train_test_split(trainingData_rs,Ydata,test_size=0.25,random_state=20)
    newYTrain = to_categorical(y_train)
    newYval = to_categorical(y_val)

    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(512, input_dim=X_train.shape[1], activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(512, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(256, input_dim=X_train.shape[1], activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(newYTrain.shape[1], activation='softmax'))

    loss='categorical_crossentropy'
    optimizer = RAdam(total_steps=5000, warmup_proportion=0.2, min_lr=1e-5)
    metrics=['accuracy']
    model.compile(loss=loss, optimizer=optimizer , metrics=metrics)
    joblib.dump({'RAdam':optimizer},'models/ANN_optimizer_'+prep+'.z')

    checkpoint=ModelCheckpoint('models/ANN_'+prep+'.h5',monitor='val_loss',verbose=0,save_best_only=True,save_weights_only=False,mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=50)
    model.fit(X_train, newYTrain, validation_data=(X_val, newYval), epochs=epochs,batch_size = batch_size, verbose=0, callbacks=[es,checkpoint]) 
    
def fitRFC(trainingData,Ydata,prep=''):
    model= RandomForestClassifier(n_estimators=100, bootstrap=True, warm_start=True).fit(trainingData, np.array(Ydata).ravel())
    joblib.dump(model,'models/RFC_'+prep+'.z') 

def fitSVC(trainingData,Ydata,prep=''):
    pca=PCA(.95)
    pca.fit(trainingData)
    model= SVC(tol=1e-3, kernel='rbf',gamma='scale',C=1).fit(pca.transform(trainingData),np.array(Ydata).ravel())
    joblib.dump(pca,'models/PCA_'+prep+'.z') 
    joblib.dump(model,'models/SVC_'+prep+'.z') 

def getAccuracy(prep,modelTypes,overwrite=False,foldScore=False,k=5):
    train_X,train_Y=np.copy(augmentedData_X[prep]),np.copy(augmentedData_Y)
    test_X,test_Y=np.copy(labelledData_X[prep]),np.copy(labelledData_Y)
    test_X,test_Y=shuffle_in_unison(test_X,test_Y)
        
    for M in modelTypes:
        if M=='CNN':
            fitCNN(train_X,train_Y,prep=prep)
            model=load_model('models/CNN_'+prep+'.h5',custom_objects=joblib.load('models/CNN_optimizer_'+prep+'.z'))
            print(M,getFfoldFitScore(model,test_X.reshape(test_X.shape[0],test_X.shape[1],1),test_Y,foldScore=foldScore,f=k))
        if M=='ANN':
            fitANN(train_X,train_Y,prep=prep)
            model=load_model('models/ANN_'+prep+'.h5',custom_objects=joblib.load('models/ANN_optimizer_'+prep+'.z'))
            print(M,getFfoldFitScore(model,test_X,test_Y,foldScore=foldScore,f=k))
        if M=='RFC':
            fitRFC(train_X,train_Y,prep=prep)
            model=joblib.load('models/RFC_'+prep+'.z')
            print(M,getFfoldFitScore(model,test_X,test_Y,foldScore=foldScore,f=k))
        if M=='SVC':
            fitSVC(train_X,train_Y,prep=prep)
            pca=joblib.load('models/PCA_'+prep+'.z') 
            model=joblib.load('models/SVC_'+prep+'.z')
            print(M,getFfoldFitScore(model,pca.transform(test_X),test_Y,foldScore=foldScore,f=k))

In [4]:
getAccuracy('_',['CNN','ANN','RFC','SVC'])

CNN (0.1307, 0.01)
ANN (0.1971, 0.008)
RFC (0.2722, 0.014)
SVC (0.0723, 0.011)


In [5]:
getAccuracy('cr',['CNN','ANN','RFC','SVC'])

CNN (0.267, 0.016)
ANN (0.2996, 0.012)
RFC (0.2748, 0.011)
SVC (0.0714, 0.004)


In [6]:
getAccuracy('crsm',['CNN','ANN','RFC','SVC'])

CNN (0.2905, 0.011)
ANN (0.3036, 0.016)
RFC (0.2525, 0.017)
SVC (0.0714, 0.001)


In [7]:
getAccuracy('crCR',['CNN','ANN','RFC','SVC'])

CNN (0.5123, 0.012)
ANN (0.5107, 0.01)
RFC (0.4036, 0.007)
SVC (0.4439, 0.012)


In [8]:
getAccuracy('crsmCR',['CNN','ANN','RFC','SVC'])

CNN (0.5398, 0.005)
ANN (0.5173, 0.011)
RFC (0.3777, 0.005)
SVC (0.445, 0.008)


In [9]:
getAccuracy('crsmSS',['CNN','ANN','RFC','SVC'])

CNN (0.7082, 0.015)
ANN (0.7506, 0.01)
RFC (0.6666, 0.011)
SVC (0.6016, 0.013)


In [10]:
getAccuracy('crsmCRSS',['CNN','ANN','RFC','SVC'])

CNN (0.8168, 0.008)
ANN (0.7905, 0.009)
RFC (0.7621, 0.007)
SVC (0.7739, 0.013)


In [11]:
getAccuracy('crsmCRSSFE',['ANN','RFC','SVC'])

ANN (0.8225, 0.005)
RFC (0.7967, 0.007)
SVC (0.8033, 0.003)
